### 1. Import library

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 2. Crawl data q_a

##### 2.1. Find elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })

    return articles_data

In [3]:
articles_data = []

for page_num in range(1, 400):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/giao-duc-dao-tao-y-te/t1/15/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

##### 2.3 Export CSV

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_giaoduc_daotao_yte = pd.DataFrame(articles_data)
df_giaoduc_daotao_yte.head(5)

,Name,Time,Question,Situation,Answer
0,Trần Đức Đỉnh,14:05 11/04/2025,Có cần chứng chỉ hành nghề điều dưỡng?,Tôi có chứng chỉ chức danh nghề nghiệp điều dư...,Bộ Y tế trả lời vấn đề này như sau:Theo quy đị...
1,Nguyễn Thị Hải,09:23 11/04/2025,Hành nghề dinh dưỡng lâm sàng cần điều kiện gì?,Tôi là bác sĩ y học dự phòng tại khoa dinh dưỡ...,"Bộ Y tế trả lời vấn đề này như sau:Hiện nay, v..."
2,Phùng Mỹ Hạnh,12:11 10/04/2025,Tính phụ cấp ưu đãi nhà giáo theo quy định nào?,"Tôi xin hỏi, việc tính phụ cấp ưu đãi theo Quy...",Bộ Giáo dục và Đào tạo trả lời vấn đề này như ...
3,Mai Ngọc Phúc,11:05 09/04/2025,Thẩm quyền cấp lại giấy phép hành nghề y,Tôi sinh sống và làm việc tại tỉnh Tây Ninh nh...,Bộ Y tế trả lời vấn đề này như sau:Hiện nay vi...
4,Đặng Phương Nhi,16:36 08/04/2025,Điều kiện cấp giấy phép hành nghề dinh dưỡng l...,Tôi là bác sĩ y học dự phòng tại một bệnh viện...,"Bộ Y tế trả lời vấn đề này như sau:Hiện nay, v..."


In [5]:
# Lưu DataFrame vào file CSV
csv_file = '../Source_Fine-Tuning/data/giaoduc_daotao_yte_data.csv'
df_giaoduc_daotao_yte.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../Source_Fine-Tuning/data/giaoduc_daotao_yte_data.csv


In [6]:
df_giaoduc_daotao_yte=pd.read_csv("../Source_Fine-Tuning/data/giaoduc_daotao_yte_data.csv",encoding='utf-8')

In [7]:
df_giaoduc_daotao_yte.shape

(1930, 6)